<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

train_data = pd.read_csv('NIFTY_50_2015-2023.csv', parse_dates=['Date'], index_col='Date')
test_data = pd.read_csv('NIFTY_50_2024.csv', parse_dates=['Date'], index_col='Date')
train_data = train_data['Close']
test_data = test_data['Close']
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

def create_dataset(dataset, look_back=60):
    X, y = [], []
    for i in range(len(dataset) - look_back - 1):
        X.append(dataset[i:(i + look_back), 0])
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 60
X_train, y_train = create_dataset(train_scaled, look_back)
X_test, y_test = create_dataset(test_scaled, look_back)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))  # Output layer
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)
predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1)) ## Invert scaling for predictions and actual values to get them back to the original scale
forecasted_prices = predictions_rescaled.flatten() #same length as the actual test data
actual_prices = y_test_rescaled.flatten()
forecasted_prices = forecasted_prices[:len(test_data) - look_back]  # Align with the test data (forecasted_prices will have one less value than the test data)
mae = mean_absolute_error(actual_prices, forecasted_prices)
rmse = sqrt(mean_squared_error(actual_prices, forecasted_prices))
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

r2 = r2_score(y_true, y_pred)
print(f'R-squared (R^2): {r2}')

# Calculate MAPE
masked_actual = np.ma.masked_array(y_true, mask=y_true==0) #Mask values where y_true is 0 to avoid division by zero.
MAPE = (np.fabs(masked_actual.filled(0) - y_pred.to_numpy())/masked_actual.filled(1)).mean() #Convert masked_actual and y_pred to NumPy arrays for compatibility.

print(f'MAPE: {MAPE}')